In [1]:
chromedriver_path = "/Users/unusual/Downloads/chromedriver-mac-arm64/chromedriver"

# 1. 타선 지원이 WHIP, ERA에 영향을 주는가?

In [3]:
import pandas as pd
import kbodata

# 경로는 사용자 환경에 맞게 조정
chromedriver_path = "/Users/unusual/Downloads/chromedriver-mac-arm64/chromedriver"

# 1. 연봉 데이터 로딩 및 선수 목록 구성
salary_df = pd.read_csv("dataset.csv")
target_players = salary_df[['Name', 'Year']].dropna().drop_duplicates()
target_players.columns = ['name', 'year']
target_players['name'] = target_players['name'].astype(str).str.strip()
target_players['year'] = target_players['year'].astype(int)

# 2. 선수 기록 수집
all_filtered_pitchers = []

for y in target_players['year'].unique():
    try:
        sched = kbodata.get_yearly_schedule(int(y), chromedriver_path)
        data = kbodata.get_game_data(sched, chromedriver_path)
        pitcher_df = kbodata.pitcher_to_DataFrame(data)
        scoreboard_df = kbodata.scoreboard_to_DataFrame(data)
        pitcher_df = pitcher_df.merge(scoreboard_df[['idx', 'year']], on='idx', how='left')

        pitcher_df['name'] = pitcher_df['name'].astype(str).str.strip()
        pitcher_df['year'] = pitcher_df['year'].astype(int)

        filtered = pitcher_df.merge(target_players, on=['name', 'year'], how='inner')
        all_filtered_pitchers.append(filtered)

    except Exception as e:
        print(f"[ERROR] {y}년 오류: {e}")

# 3. 결합 및 저장
final_pitcher_df = pd.concat(all_filtered_pitchers, ignore_index=True)
final_pitcher_df.to_csv("filtered_pitcher_data.csv", index=False)

in progress: 100%|████████████████████████████████████████████████████████████████████████████████| 12/12 [00:13<00:00,  1.16s/it]


[ERROR] 2021년 오류: Message: 



in progress:  93%|████████████████████████████████████████████████████████████████████████▏     | 802/866 [15:12<01:12,  1.14s/it]


[ERROR] 2024년 오류: '중파'


in progress: 100%|██████████████████████████████████████████████████████████████████████████████| 737/737 [06:25<00:00,  1.91it/s]


In [7]:
# 전처리: 연봉 문자열 숫자로 변환
salary_df = salary_df.dropna(subset=['Year', 'Name'])
salary_df['salary'] = salary_df['연봉'].astype(str).str.replace(',', '').astype(float)
salary_df['year'] = salary_df['Year'].astype(int)
salary_df['name'] = salary_df['Name'].astype(str).str.strip()

# 병합
merged_df = final_pitcher_df.merge(salary_df[['name', 'year', 'salary', 'FA']], on=['name', 'year'], how='left')

In [8]:
# InGameRS 포함 데이터 저장# IP (이닝 소화): 3으로 나눠서 계산
merged_df['IP'] = merged_df['inning'].astype(int) // 3 + (merged_df['inning'].astype(int) % 3) / 3

# ERA 계산 (earnedrun / IP * 9), 단 IP가 0이면 NaN
merged_df['ERA'] = merged_df.apply(lambda row: (row['earnedrun'] / row['IP'] * 9) if row['IP'] > 0 else None, axis=1)
pitcher_df.to_csv(f"pitcher_with_rs_{y}.csv", index=False)

In [9]:
# scoreboard_df에서 InGameRS 추출
rs_df = scoreboard_df[['idx', 'team', 'r']].rename(columns={'r': 'InGameRS'})

# 득점 병합
merged_df = merged_df.merge(rs_df, on=['idx', 'team'], how='left')

In [10]:
import statsmodels.api as sm

# 결측 제거
reg_df = merged_df.dropna(subset=['ERA', 'InGameRS', 'IP', 'FA'])

X = sm.add_constant(reg_df[['InGameRS', 'IP', 'FA']])
y = reg_df['ERA']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    ERA   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                 -0.164
Method:                 Least Squares   F-statistic:                    0.2270
Date:                Mon, 26 May 2025   Prob (F-statistic):              0.801
Time:                        09:02:54   Log-Likelihood:                -15.333
No. Observations:                  12   AIC:                             36.67
Df Residuals:                       9   BIC:                             38.12
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4720      1.022      1.441      0.1

/Users/unusual/miniconda3/envs/capstone/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  res = hypotest_fun_out(*samples, **kwds)
/Users/unusual/miniconda3/envs/capstone/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


## 타선 지원과 공격성 지표

In [11]:
# IP 계산 (이닝: 3단위로 기록되어 있음)
merged_df['IP'] = merged_df['inning'].astype(int) // 3 + (merged_df['inning'].astype(int) % 3) / 3

# K9 계산
merged_df['K9'] = merged_df.apply(lambda row: (row['strikeout'] / row['IP'] * 9) if row['IP'] > 0 else None, axis=1)

In [12]:
import statsmodels.api as sm

df = merged_df.dropna(subset=['K9', 'InGameRS', 'IP'])

X = sm.add_constant(df[['InGameRS', 'IP']])
y = df['K9']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     K9   R-squared:                       0.410
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     3.122
Date:                Mon, 26 May 2025   Prob (F-statistic):             0.0934
Time:                        09:07:47   Log-Likelihood:                -13.506
No. Observations:                  12   AIC:                             33.01
Df Residuals:                       9   BIC:                             34.47
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3197      0.877      0.364      0.7

/Users/unusual/miniconda3/envs/capstone/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  res = hypotest_fun_out(*samples, **kwds)


In [13]:
merged_df

,idx,name,team,mound,inning,result,strikeout,dead4ball,losescore,earnedrun,...,homerun,battednum,batternum,year,salary,FA,IP,ERA,InGameRS,K9
0,20220312004,김도현,한화,0,20,L,1,0,0,0,...,0,7,7,2022,3500.0,0.0,6.666667,0.000000,NaN,1.350000
1,20220312005,손주영,LG,1,30,L,4,1,0,0,...,0,10,11,2022,3800.0,0.0,10.000000,0.000000,NaN,3.600000
2,20220315005,임찬규,LG,1,22,L,2,2,0,0,...,0,9,11,2022,20000.0,0.0,7.333333,0.000000,NaN,2.454545
3,20220317001,최원준,두산,1,22,L,1,1,2,2,...,0,12,13,2022,34000.0,0.0,7.333333,2.454545,NaN,1.227273
4,20220317004,김도현,한화,0,22,L,0,1,4,4,...,0,13,14,2022,3500.0,0.0,7.333333,4.909091,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,20250404005,송승기,LG,1,50,W,4,3,2,2,...,0,22,25,2025,3500.0,0.0,16.666667,1.080000,8.0,2.160000
923,20250411005,송승기,LG,1,60,L,5,3,2,2,...,0,20,25,2025,3500.0,0.0,20.000000,0.900000,5.0,2.250000
924,20250417005,송승기,LG,1,42,L,4,2,4,4,...,1,20,22,2025,3500.0,0.0,14.000000,2.571429,3.0,2.571429
925,20250423005,송승기,LG,1,60,W,7,3,0,0,...,0,19,22,2025,3500.0,0.0,20.000000,0.000000,3.0,3.150000


In [14]:
# 1. 경기별 승리 팀 계산
# 예: scoreboard_df → [idx, team, r]
scoreboard_result = scoreboard_df[['idx', 'team', 'r']].copy()

# 각 경기(idx)에서 r이 가장 높은 팀을 승리팀으로 판단
scoreboard_result['r'] = scoreboard_result['r'].astype(float)
winner_df = scoreboard_result.loc[scoreboard_result.groupby('idx')['r'].idxmax()]
winner_df = winner_df.rename(columns={'team': 'winner_team'})[['idx', 'winner_team']]

# 2. pitcher 데이터에 병합
merged_df = merged_df.merge(winner_df, on='idx', how='left')

# 3. 승리 여부 부여
merged_df['W'] = (merged_df['team'] == merged_df['winner_team']).astype(int)

In [15]:
import statsmodels.api as sm

df = merged_df.dropna(subset=['ERA', 'W'])

X = sm.add_constant(df[['W']])
y = df['ERA']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    ERA   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.3513
Date:                Mon, 26 May 2025   Prob (F-statistic):              0.553
Time:                        09:11:30   Log-Likelihood:                -3072.6
No. Observations:                 921   AIC:                             6149.
Df Residuals:                     919   BIC:                             6159.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.1221      0.226      9.396      0.0

In [16]:
df1 = merged_df.dropna(subset=['ERA', 'InGameRS'])

X1 = sm.add_constant(df1[['InGameRS']])
y1 = df1['ERA']

model1 = sm.OLS(y1, X1).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                    ERA   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.096
Method:                 Least Squares   F-statistic:                   0.03393
Date:                Mon, 26 May 2025   Prob (F-statistic):              0.858
Time:                        09:14:36   Log-Likelihood:                -15.608
No. Observations:                  12   AIC:                             35.22
Df Residuals:                      10   BIC:                             36.19
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8829      0.457      1.933      0.0

/Users/unusual/miniconda3/envs/capstone/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  res = hypotest_fun_out(*samples, **kwds)


In [17]:
df2 = merged_df.dropna(subset=['salary', 'InGameRS', 'ERA'])

X2 = sm.add_constant(df2[['InGameRS', 'ERA']])
y2 = df2['salary']

model2 = sm.OLS(y2, X2).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                        -inf
Model:                            OLS   Adj. R-squared:                   -inf
Method:                 Least Squares   F-statistic:                    -4.500
Date:                Mon, 26 May 2025   Prob (F-statistic):               1.00
Time:                        09:14:49   Log-Likelihood:                 278.75
No. Observations:                  12   AIC:                            -551.5
Df Residuals:                       9   BIC:                            -550.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3500.0000   1.25e-11   2.79e+14      0.0

/Users/unusual/miniconda3/envs/capstone/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  res = hypotest_fun_out(*samples, **kwds)
/Users/unusual/miniconda3/envs/capstone/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [18]:
import statsmodels.api as sm

df = merged_df.dropna(subset=['IP', 'ERA', 'K9', 'W'])

X = sm.add_constant(df[['ERA', 'K9', 'W']])
y = df['IP']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     IP   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     45.32
Date:                Mon, 26 May 2025   Prob (F-statistic):           2.60e-27
Time:                        09:16:57   Log-Likelihood:                -3058.0
No. Observations:                 921   AIC:                             6124.
Df Residuals:                     917   BIC:                             6143.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.7267      0.296     56.534      0.0

In [19]:
import statsmodels.api as sm

df = merged_df.dropna(subset=['W', 'InGameRS'])

X = sm.add_constant(df[['InGameRS']])
y = df['W']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      W   R-squared:                        -inf
Model:                            OLS   Adj. R-squared:                   -inf
Method:                 Least Squares   F-statistic:                    -10.00
Date:                Mon, 26 May 2025   Prob (F-statistic):               1.00
Time:                        09:17:54   Log-Likelihood:                 418.73
No. Observations:                  12   AIC:                            -833.5
Df Residuals:                      10   BIC:                            -832.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0000   8.72e-17   1.15e+16      0.0

/Users/unusual/miniconda3/envs/capstone/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  res = hypotest_fun_out(*samples, **kwds)
/Users/unusual/miniconda3/envs/capstone/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [21]:
winner_df = scoreboard_df.loc[scoreboard_df.groupby('idx')['r'].idxmax()]
winner_df = winner_df.rename(columns={'team': 'winner_team'})[['idx', 'winner_team']]

## 6번

In [29]:
filtered_pitchers = pd.read_csv("filtered_pitcher_data.csv")

In [30]:
# 연도별로 scoreboard_df 다시 불러오기
all_scoreboards = []

for y in filtered_pitchers['year'].unique():
    sched = kbodata.get_yearly_schedule(int(y), chromedriver_path)
    data = kbodata.get_game_data(sched, chromedriver_path)
    scoreboard_df = kbodata.scoreboard_to_DataFrame(data)
    all_scoreboards.append(scoreboard_df[['idx', 'team', 'r']])

scoreboard_df = pd.concat(all_scoreboards, ignore_index=True)

in progress: 100%|██████████████████████████████████████████████████████████████████████████████| 737/737 [07:25<00:00,  1.65it/s]


In [54]:
starting_pitchers['idx'] = starting_pitchers['idx'].astype(str)
winner_rows['idx'] = winner_rows['idx'].astype(str)

In [55]:
starting_pitchers = starting_pitchers.merge(winner_rows, on='idx', how='left')
starting_pitchers['W'] = (starting_pitchers['team'] == starting_pitchers['winner_team']).astype(int)

In [56]:
# 3. 선발투수만 추출
starting_pitchers = filtered_pitchers[filtered_pitchers['mound'] == 1].copy()
starting_pitchers['idx'] = starting_pitchers['idx'].astype(str)

# 승리팀 추출
winner_rows = scoreboard_df.loc[scoreboard_df.groupby('idx')['r'].idxmax()].copy()
winner_rows.rename(columns={'team': 'winner_team'}, inplace=True)
winner_rows = winner_rows[['idx', 'winner_team']]
winner_rows['idx'] = winner_rows['idx'].astype(str)  # ← 이거 안 하면 오류

# 병합 및 W 생성
starting_pitchers = starting_pitchers.merge(winner_rows, on='idx', how='left')
starting_pitchers['W'] = (starting_pitchers['team'] == starting_pitchers['winner_team']).astype(int)

In [57]:
# 4. 타선 득점 InGameRS 계산
ingamers_rows = scoreboard_df[['idx', 'team', 'r']].copy()
ingamers_rows.rename(columns={'r': 'InGameRS'}, inplace=True)
ingamers_rows['idx'] = ingamers_rows['idx'].astype(str)

# 병합
starting_pitchers = starting_pitchers.merge(ingamers_rows, on=['idx', 'team'], how='left')

In [59]:
# scoreboard_df와 starting_pitchers의 team 컬럼 정제
scoreboard_df['team'] = scoreboard_df['team'].str.strip()
starting_pitchers['team'] = starting_pitchers['team'].str.strip()
winner_rows['winner_team'] = winner_rows['winner_team'].str.strip()

In [60]:
# 5. 회귀분석
df = starting_pitchers[['W', 'InGameRS']].dropna()
X = sm.add_constant(df[['InGameRS']])
y = df['W']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      W   R-squared:                        -inf
Model:                            OLS   Adj. R-squared:                   -inf
Method:                 Least Squares   F-statistic:                    -519.0
Date:                Mon, 26 May 2025   Prob (F-statistic):               1.00
Time:                        13:05:16   Log-Likelihood:                 18104.
No. Observations:                 521   AIC:                        -3.620e+04
Df Residuals:                     519   BIC:                        -3.620e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0000    1.5e-17   6.68e+16      0.0

/Users/unusual/miniconda3/envs/capstone/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [61]:
print(starting_pitchers['W'].value_counts())

W
1    521
0    202
Name: count, dtype: int64
